In [ ]:
# Imports

!pip install imageai
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import PIL.Image
from PIL import ImageOps
import PIL
import pathlib
import matplotlib.pyplot as plt
import datetime
import tensorboard
import IPython
import sklearn
import cv2
import subprocess
import sys
from imageai.Classification import ImageClassification
from google.colab import drive

In [ ]:
prediction = ImageClassification()

drive.mount('/content/drive')

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = True, 
    weights = "imagenet"                  
)

saving_path = pathlib.Path('/content/drive/MyDrive/imageai/SavedBaseModel.h5')

base_model.save(saving_path)

model_path = pathlib.Path('/content/drive/MyDrive/imageai/SavedBaseModel.h5')

prediction.setModelTypeAsInceptionV3()

prediction.setModelPath("/content/drive/MyDrive/imageai/SavedBaseModel.h5")

prediction.loadModel()

predictions, probabilities = prediction.classifyImage("/content/drive/MyDrive/License_Plate_Recognition/LicPlateImages/10.png", result_count=10)

for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)
